# Import Dependensi

Package

In [1]:
import pandas as pd
import numpy as np

Modul Perhitungan ET

In [2]:
from modul.gravimetri import gravimetri_numpy as gravimetri
from modul.bc_model import bc_model
from modul.hs_model import hs_model
from modul.pm_model import pm_model

Variabel

In [3]:
from variabel import variabel as var

# Import Dataset

In [4]:
# Import file excel
dataset_primer = 'data/dataset_primer_2_processed.xlsx'
dataset_sekunder = 'data/dataset_sekunder_2.xlsx'

# Membaca semua sheet dari file Excel
data_primer = pd.ExcelFile(dataset_primer)
data_sekunder = pd.ExcelFile(dataset_sekunder)

# Fungsi-Fungsi

In [5]:
# Menghitung luas area di dalam variabel
def hitung_luas_media(sisi):
        cm_to_mm = sisi[0] * 10
        luas = (cm_to_mm ** 2)
        return luas              # Hasil dalam satuan mm

In [6]:
# Menghitung tanggal pengambilan data
def hitung_tanggal_data(day):
    day = int(day.split()[1])

    tahun = '2024'

    if day > 10:
        bulan = '11'
    else:
        bulan = '10'
    
    def hitung_tanggal(input_day):
        # Jika x antara 1 dan 31, y adalah x + 20, tetapi melingkar kembali ke awal setelah 31
        return (input_day + 21) % 31 if (input_day + 21) % 31 != 0 else 31

    hari = hitung_tanggal(day)
    hari = f"{hari:02d}"

    tanggal = (f"{tahun}-{bulan}-{hari}")

    return tanggal

# ET Model Gravimetri/Penimbangan

Perhitungan Gravimetri

In [7]:
hasil_gravimetri = []

for sheet_name in data_primer.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_primer = data_primer.parse(sheet_name)
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    for kode in var.sisi_media_tanaman.keys():
        kode_berat = f'berat_{kode}'
        kode_irigasi = f'supply_air_{kode}'
        
        # Buat array untuk variabel yang akan digunakan
        waktu = df_data_primer['waktu'].array
        berat = df_data_primer[kode_berat].array                            # Satuan kg
        irigasi_liter = df_data_primer[kode_irigasi].fillna(0).array        # Satuan liter
        curah_hujan = df_data_sekunder['curah_hujan'].array                 # Satuan mm

        # Hitung luas alas media tanaman
        luas_media = hitung_luas_media(var.sisi_media_tanaman[kode])    # Satuan mm^2

        # Ubah irigasi ke satuan mm
        irigasi_mm = (irigasi_liter * 1e6) / (luas_media)               # Satuan mm

        # Ubah berat ke satuan volume mm^3
        volume = berat * 1e6                                            # Satuan mm^3

        # Hitung total supply air
        supply_air = irigasi_mm + curah_hujan                           # Satuan mm

        perubahan_berat, perubahan_et, et_total = gravimetri(berat=volume,
                                                            waktu=waktu,
                                                            luas=luas_media,
                                                            supply_air=supply_air
                                                            )

        # Simpan hasil ke list
        hasil_gravimetri.append({
            "Num Day": sheet_name,
            "Kode": kode,
            "ET Total": et_total
        })

# Buat DataFrame dari hasil
df_hasil_gravimetri = pd.DataFrame(hasil_gravimetri)

# Pivot tabel dengan "Day" sebagai index dan "Kode" sebagai kolom
df_hasil_gravimetri = df_hasil_gravimetri.pivot(index="Num Day", columns="Kode", values="ET Total")


ValueError: Worksheet named 'day 05' not found

In [54]:
df_hasil_gravimetri.to_csv('hasil/ET Aktual Gravimetri.csv', index=True)
df_hasil_gravimetri.head()

Kode,A1,A2,A3,A4,B1,B2,B3,B4,C1,C2,...,D3,D4,E1,E2,E3,E4,F1,F2,F3,F4
Num Day,,,,,,,,,,,,,,,,,,,,,
day 06,2.0140,8.1900,6.5260,10.334,10.207778,7.541111,7.096667,6.096667,8.225918,4.078980,...,8.69250,3.26750,3.091728,6.301605,2.380617,6.439877,1.4900,2.658000,1.930000,4.378000
day 07,158.5024,199.0432,251.6384,226.640,184.697778,244.144444,203.784444,210.231111,239.103673,250.360816,...,280.03750,259.02625,282.361481,320.282469,298.488889,285.489383,347.1304,380.280800,368.587200,379.207200
day 08,73.0240,97.8560,119.2320,106.080,85.777778,113.444444,96.244444,100.488889,114.253061,118.546939,...,135.86250,140.06250,140.138272,155.476543,146.656790,284.827090,171.3600,189.048000,186.240000,189.656000
day 09,159.9020,203.8956,250.7180,226.766,182.212222,244.376667,203.978889,209.725556,234.020204,241.220204,...,279.22125,265.30625,283.611235,318.799383,302.219630,286.859630,342.9356,378.643486,368.004171,378.865429
day 10,73.7060,96.2020,308.9380,106.154,82.205556,113.227778,93.672222,98.538889,112.470408,118.886735,...,135.35000,127.22500,138.605556,155.079630,147.198148,141.746296,172.0740,187.602000,178.338000,181.674000


# ET Model Blaney-Criddle

In [55]:
hasil_bc_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)

    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")
    
    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])
    # print(f"Suhu udara {sheet_name}: {suhu_udara}")

    # Menghitung rata-rata suhu udara
    mean_suhu_udara = np.mean(suhu_udara)
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")

    # Menghitung ET dengan model Blaney-Criddle
    et_model_bc = bc_model(var.persentase_hari_terang, mean_suhu_udara)
    # print(f"ET Blaney-Criddle: {et_model_bc}\n")

    # Simpan hasil ke list
    hasil_bc_model.append({
        "Num Day": sheet_name,
        "Tanggal": tanggal_data,
        "Mean Suhu Udara": mean_suhu_udara,
        "ET Total": et_model_bc
    })

# Buat DataFrame dari hasil
df_hasil_bc = pd.DataFrame(hasil_bc_model)

In [56]:
df_hasil_bc.to_csv('hasil/Estimasi ET Blaney-Criddle.csv', index=False)
df_hasil_bc.head()

,Num Day,Tanggal,Mean Suhu Udara,ET Total
0,day 06,2024-10-27,31.77875,6.106921
1,day 07,2024-10-28,32.63000,6.212646
2,day 08,2024-10-29,32.22250,6.162035
3,day 09,2024-10-30,31.17125,6.031469
4,day 10,2024-10-31,29.55625,5.830886


# ET Model Hargreaves-Samani

In [57]:
hasil_hs_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")

    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])

    # Menghitung rata-rata, max, min, dan rentang suhu udara
    max_suhu_udara = np.max(suhu_udara)
    # print(f"Maksimal suhu udara: {max_suhu_udara}")
    min_suhu_udara = np.min(suhu_udara)
    # print(f"Minimal suhu udara: {min_suhu_udara}")
    mean_suhu_udara = (max_suhu_udara + min_suhu_udara) / 2
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")

    # Menghitung ET dengan model Hargreaves-Samani
    radiasi_ekstrateresterial, et_model_hs, et_model_hs_modification = hs_model(t_max = max_suhu_udara, 
                                                                                t_min = min_suhu_udara, 
                                                                                t_avg = mean_suhu_udara, 
                                                                                latitude = var.latitude, 
                                                                                input_date = tanggal_data)
    # print(f"ET Hargreaves-Samani: {et_model_hs}")
    # print(f"Radiasi atmosfer: {radiasi_ekstrateresterial}")

    # Simpan hasil ke list
    hasil_hs_model.append({
        "Num Day": sheet_name,
        "Tanggal": tanggal_data,
        "Max Suhu Udara": max_suhu_udara,
        "Min Suhu Udara": min_suhu_udara,
        "Mean Suhu Udara": mean_suhu_udara,
        "Radiasi Extrateresterial": radiasi_ekstrateresterial,
        "ET Total": et_model_hs,
    })

# Buat DataFrame dari hasil
df_hasil_hs = pd.DataFrame(hasil_hs_model)

In [58]:
df_hasil_hs.to_csv('hasil/Estimasi ET Hargreaves-Samani.csv', index=False)
df_hasil_hs.head()

,Num Day,Tanggal,Max Suhu Udara,Min Suhu Udara,Mean Suhu Udara,Radiasi Extrateresterial,ET Total
0,day 06,2024-10-27,34.18,25.99,30.085,15.798661,4.979544
1,day 07,2024-10-28,35.19,26.08,30.635,15.804507,5.314070
2,day 08,2024-10-29,34.46,26.55,30.505,15.809836,4.940098
3,day 09,2024-10-30,32.96,26.74,29.850,15.814667,4.322609
4,day 10,2024-10-31,30.95,25.83,28.390,15.819018,3.802684


# ET Model Penman-Monteith

In [59]:
hasil_pm_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")

    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])
    kecepatan_angin = np.array(df_data_sekunder['kecepatan_angin'])
    kelembapan_relatif = np.array(df_data_sekunder['kelembapan_relatif'])
    
    # Menghitung rata-rata kecepatan angin dan kelembapan relatif
    mean_kecepatan_angin = np.mean(kecepatan_angin)
    # print(f"Rata-rata kecepatan angin: {mean_kecepatan_angin}")
    mean_kelembapan_relatif = np.mean(kelembapan_relatif)
    # print(f"Rata-rata kelembapan relatif: {mean_kelembapan_relatif}")

    # Menghitung rata-rata, max, min, dan rentang suhu udara
    max_suhu_udara = np.max(suhu_udara)
    # print(f"Maksimal suhu udara: {max_suhu_udara}")
    min_suhu_udara = np.min(suhu_udara)
    # print(f"Minimal suhu udara: {min_suhu_udara}")
    mean_suhu_udara = (max_suhu_udara + min_suhu_udara) / 2
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")
    
    # Menghitung ET dengan model Penman-monteith
    radiasi_ekstrateresterial, et_model_pm, total_et_model_pm = pm_model(RH = mean_kelembapan_relatif, 
                                                                         u_2 = kecepatan_angin, 
                                                                         T_hr = suhu_udara, 
                                                                         T_avg = mean_suhu_udara, 
                                                                         T_max = max_suhu_udara, 
                                                                         T_min = min_suhu_udara, 
                                                                         altitude = var.altitude, 
                                                                         latitude = var.latitude, 
                                                                         input_date = tanggal_data)
    # print(f"ET Penman-monteith {sheet_name}: {total_et_model_pm}")
    # print(f"Radiasi atmosfer: {radiasi_ekstrateresterial}")

    # Simpan hasil ke list
    hasil_pm_model.append({
        "Num Day": sheet_name,
        "Tanggal": tanggal_data,
        "Max Suhu Udara": max_suhu_udara,
        "Min Suhu Udara": min_suhu_udara,
        "Mean Suhu Udara": mean_suhu_udara,
        "Mean Kecepatan Angin": mean_kecepatan_angin,
        "Mean Kelembapan Relatif": mean_kelembapan_relatif,
        "Radiasi Extrateresterial": radiasi_ekstrateresterial,
        "ET Total": total_et_model_pm
    })

# Buat DataFrame dari hasil
df_hasil_pm = pd.DataFrame(hasil_pm_model)

In [60]:
df_hasil_pm.to_csv('hasil/Estimasi ET Penman-Monteith.csv', index=False)
df_hasil_pm.head()

,Num Day,Tanggal,Max Suhu Udara,Min Suhu Udara,Mean Suhu Udara,Mean Kecepatan Angin,Mean Kelembapan Relatif,Radiasi Extrateresterial,ET Total
0,day 06,2024-10-27,34.18,25.99,30.085,2.74875,45.59375,15.798661,3.876762
1,day 07,2024-10-28,35.19,26.08,30.635,2.78875,42.54750,15.804507,4.293068
2,day 08,2024-10-29,34.46,26.55,30.505,3.95125,44.36125,15.809836,4.737096
3,day 09,2024-10-30,32.96,26.74,29.850,4.02250,50.61750,15.814667,3.990299
4,day 10,2024-10-31,30.95,25.83,28.390,2.42250,57.59625,15.819018,2.516000
